In [1]:
import glob
import numpy as np
csv_files = (glob.glob("C:/Users/Vishnu Kumar/Documents/IPL/raw/*info.csv")) 

In [2]:
import pandas as pd


df_append = pd.DataFrame()

for file in csv_files:
            df_temp = pd.read_csv(file,sep='漢ழ்字', engine='python',header=None)
            df_temp['ID'] = file
            df_temp = df_temp[(df_temp[0].str.startswith('info,toss_winner')) | (df_temp[0].str.startswith('info,toss_decision'))]
            df_append = pd.concat([df_append,df_temp])

df_append = df_append.reset_index()
df_append[['info', 'attrib','value']] = df_append[0].str.split(",", expand = True)
df_append = df_append.drop(columns=['index',0,'info'])
df_append['ID'] = df_append['ID'].str[40:-9]
toss_winner = df_append[df_append['attrib']=='toss_winner'].copy()
toss_decision = df_append[df_append['attrib']=='toss_decision'].copy()
toss_winner['ID'] = toss_winner['ID'].astype('int')
toss_decision['ID'] = toss_decision['ID'].astype('int')

In [17]:
all_matches = pd.read_csv("C:/Users/Vishnu Kumar/Documents/IPL/raw/all_matches.csv")
# all_matches
all_matches = pd.merge(all_matches, toss_winner, how="left" ,left_on = "match_id", right_on = "ID" )
all_matches = all_matches.drop(columns=["ID", "attrib"])
all_matches = all_matches.rename(columns={"value": "toss_winner"})
all_matches = pd.merge(all_matches, toss_decision, how="left" ,left_on = "match_id", right_on = "ID" )
all_matches = all_matches.drop(columns=["ID", "attrib"])
all_matches = all_matches.rename(columns={"value": "toss_decision"})
all_matches = all_matches[all_matches['innings']<=2]
all_matches['season'] = all_matches['season'].replace(['2007/08', '2009', '2009/10', '2011' ,'2012','2018', '2019', '2020/21', '2021'], [2008,2009,2010,2011,2012,2018,2019,2020,2021])
all_matches['toss_win_batting'] = all_matches.apply(lambda x: 1 if x['toss_winner'] == x['batting_team'] else 0, axis=1)
all_matches['toss_decision'] = all_matches.apply(lambda x: 1 if x['toss_decision'] == 'bat' else 0, axis=1)
all_matches['player_dismissed'] = all_matches.apply(lambda x: 1 if x['player_dismissed'] == x['striker'] else 0, axis=1)
all_data = all_matches.copy()
# all_matches = pd.concat([all_matches, toss_winner], axis=1, join='outer', join_axes=[match_id,ID])

C:\Users\Vishnu Kumar\AppData\Local\Temp\ipykernel_8340\579395103.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  all_matches = pd.read_csv("C:/Users/Vishnu Kumar/Documents/IPL/raw/all_matches.csv")


In [18]:
all_matches['ball'] = (all_matches['ball'].astype('int')*6)+((all_matches['ball'] - all_matches['ball'].astype('int'))*10)

In [19]:
all_matches = all_matches.fillna(0)

In [20]:
all_matches['byes'] = all_matches['byes'] + all_matches['legbyes']
all_matches['extras'] = all_matches['extras'] - all_matches['byes']

In [21]:
all_matches = all_matches.drop(columns=['match_id','season','start_date','innings','batting_team','non_striker','bowling_team','player_dismissed','other_wicket_type','toss_winner','toss_decision','toss_win_batting','wides','noballs','legbyes','penalty','other_player_dismissed'])

In [22]:
all_matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225793 entries, 0 to 225953
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   venue         225793 non-null  object 
 1   ball          225793 non-null  float64
 2   striker       225793 non-null  object 
 3   bowler        225793 non-null  object 
 4   runs_off_bat  225793 non-null  int64  
 5   extras        225793 non-null  float64
 6   byes          225793 non-null  float64
 7   wicket_type   225793 non-null  object 
dtypes: float64(3), int64(1), object(4)
memory usage: 15.5+ MB


In [40]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Embedding
from keras.optimizers import Adam
from keras.losses import mean_squared_error
from keras.layers import Flatten

# Load the dataset
df = all_matches.copy()

# one-hot encode categorical features
df = pd.get_dummies(df, columns=['wicket_type'])
df = pd.get_dummies(df, columns=['venue', 'ball', 'striker','bowler'])
df = df.fillna(0)

cols_to_normalize = df.columns[:]
df[cols_to_normalize] = (df[cols_to_normalize]-df[cols_to_normalize].min())/(df[cols_to_normalize].max()-df[cols_to_normalize].min())

# split the dataset into train and test sets
train_set = df.sample(frac=0.8, random_state=42)
test_set = df.drop(train_set.index)

# extract the features and labels
X_train = train_set.iloc[:, 15:].values
y_train = train_set.iloc[:, 0:14].values
X_test = test_set.iloc[:, 15:].values
y_test = test_set.iloc[:, 0:14].values

In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Define the model architecture
model = Sequential()
model.add(Dense(64, input_shape=(X_train_scaled.shape[1],), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(14, activation='linear'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test))

# Evaluate the model
score = model.evaluate(X_test_scaled, y_test)
print('Test loss:', score)


Epoch 1/50
5645/5645 [==============================] - 22s 4ms/step - loss: 0.0501 - val_loss: 0.0125
Epoch 2/50
5645/5645 [==============================] - 19s 3ms/step - loss: 0.0124 - val_loss: 0.0122
Epoch 3/50
5645/5645 [==============================] - 19s 3ms/step - loss: 0.0122 - val_loss: 0.0121
Epoch 4/50
5645/5645 [==============================] - 19s 3ms/step - loss: 0.0121 - val_loss: 0.0121
Epoch 5/50
5645/5645 [==============================] - 19s 3ms/step - loss: 0.0120 - val_loss: 0.0121
Epoch 6/50
5645/5645 [==============================] - 19s 3ms/step - loss: 0.0120 - val_loss: 0.0121
Epoch 7/50
5645/5645 [==============================] - 19s 3ms/step - loss: 0.0120 - val_loss: 0.0122
Epoch 8/50
5645/5645 [==============================] - 20s 3ms/step - loss: 0.0119 - val_loss: 0.0122
Epoch 9/50
5645/5645 [==============================] - 19s 3ms/step - loss: 0.0119 - val_loss: 0.0122
Epoch 10/50
5645/5645 [==============================] - 19s 3ms/step - l

In [51]:
import sklearn.metrics as metrics

# predict on test data
y_pred = model.predict(X_test_scaled)

# evaluate the model
test_loss = model.evaluate(X_test_scaled, y_test, verbose=0)
# accuracy = metrics.accuracy_score(y_test, y_pred.round())

print(f"Loss: {test_loss:.4f}, Accuracy: {100:.2f}%")


1412/1412 [==============================] - 3s 2ms/step
Loss: 0.0210, Accuracy: 100.00%


In [47]:
print(y_test.shape)
print(np.unique(y_test))


(45159, 14)
[0.         0.14285714 0.16666667 0.2        0.28571429 0.33333333
 0.4        0.42857143 0.5        0.57142857 0.6        0.66666667
 0.71428571 0.8        0.83333333 1.        ]


In [49]:
print(np.unique(y_pred))


[-50.97171   -12.550868   -3.029262  ...   7.751359    7.8287644
  46.770107 ]
